In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance,PointStruct
from img2vec_pytorch import Img2Vec
import glob
import os
import csv
from PIL import Image

In [4]:
client = QdrantClient(url="http://localhost:6333")
model = Img2Vec(model='efficientnet_b0')


/Users/lucas/Developer/tcc/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/lucas/Developer/tcc/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /Users/lucas/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 48.2MB/s]


In [5]:
client.recreate_collection(
    collection_name="building-patterns",
    vectors_config=VectorParams(size=1280, distance=Distance.COSINE),
)


True

In [6]:
def generate_embedding(filename):
    img = Image.open(filename).convert("RGB")
    vec = model.get_vec(img=img)
    return vec.tolist()

In [7]:
with open("/Users/lucas/Developer/tcc/datasets/full/sb_training.csv", 'r', encoding='utf-8') as all_images:
    reader = csv.DictReader(all_images)
    lines = list(reader)

lines

[{'foto': 'https://geo-criciuma-fotos.s3-us-east-2.amazonaws.com/files/imoveis/3145019/33.1_1570625004221.jpg',
  'class_id': '3',
  'padrao_construtivo': 'Médio'},
 {'foto': 'https://geo-criciuma-fotos.s3-us-east-2.amazonaws.com/files/imoveis/3336835/05_1599845566225.jpg',
  'class_id': '3',
  'padrao_construtivo': 'Médio'},
 {'foto': 'https://geo-criciuma-fotos.s3-us-east-2.amazonaws.com/files/imoveis/3085860/17.1_1569948160184.jpg',
  'class_id': '3',
  'padrao_construtivo': 'Médio'},
 {'foto': 'https://geo-criciuma-fotos.s3-us-east-2.amazonaws.com/files/imoveis/3081750/1_1569339287149.jpg',
  'class_id': '4',
  'padrao_construtivo': 'Médio Baixo'},
 {'foto': 'https://geo-criciuma-fotos.s3-us-east-2.amazonaws.com/files/imoveis/3336614/04_1596823991068.jpg',
  'class_id': '4',
  'padrao_construtivo': 'Médio Baixo'},
 {'foto': 'https://geo-criciuma-fotos.s3-us-east-2.amazonaws.com/files/imoveis/3063640/datecamera0427134338_1588783572405.jpg',
  'class_id': '4',
  'padrao_construtivo':

In [8]:
dir_name = "/Users/lucas/Developer/tcc/datasets/full/images_jpg"
image_glob = glob.glob(dir_name + '/*' + '.jpeg')
points = list()
id = 1
print(len(lines))


1136


In [9]:


for image in image_glob:
     for line in lines:
          try:
               if str(os.path.basename(image)).replace('.jpeg','') in str(line['foto']):
                    embedding = generate_embedding(image)
                    points.append(PointStruct(id=id, vector=embedding, payload={'class_id': line['class_id'], 
                                                                                'building_pattern' : line['padrao_construtivo']}
                                              )
                                  )
                    print(f"Ponto com id {id} inserido")
                    id+=1
                    if len(points) == 50:
                         operation_info = client.upsert(
                         collection_name='building-patterns',
                         wait=True,
                         points=points
                    )
                         print(operation_info)
                         points.clear()
                         print(points)
                    
          except Exception as error:
               print(error)
if len(points) > 0:
     operation_info = client.upsert(
          collection_name='building-patterns',
          wait=True,
          points=points
     )
     print(operation_info)


Ponto com id 1 inserido
Ponto com id 2 inserido
Ponto com id 3 inserido
Ponto com id 4 inserido
Ponto com id 5 inserido
Ponto com id 6 inserido
Ponto com id 7 inserido
Ponto com id 8 inserido
Ponto com id 9 inserido
Ponto com id 10 inserido
Ponto com id 11 inserido
Ponto com id 12 inserido
Ponto com id 13 inserido
Ponto com id 14 inserido
Ponto com id 15 inserido
Ponto com id 16 inserido
Ponto com id 17 inserido
Ponto com id 18 inserido
Ponto com id 19 inserido
Ponto com id 20 inserido
Ponto com id 21 inserido
Ponto com id 22 inserido
Ponto com id 23 inserido
Ponto com id 24 inserido
Ponto com id 25 inserido
Ponto com id 26 inserido
Ponto com id 27 inserido
Ponto com id 28 inserido
Ponto com id 29 inserido
Ponto com id 30 inserido
Ponto com id 31 inserido
Ponto com id 32 inserido
Ponto com id 33 inserido
Ponto com id 34 inserido
Ponto com id 35 inserido
Ponto com id 36 inserido
Ponto com id 37 inserido
Ponto com id 38 inserido
Ponto com id 39 inserido
Ponto com id 40 inserido
Ponto com